In [1]:
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from PIL import Image
import PIL
import os, glob, sys, shutil
#import vissl
import tensorboard
#import apex
import torch
#from vissl.utils.io import load_file
import xarray
import math
import cv2
import scipy
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
#from vissl.utils.io import save_file
from matplotlib.image import NonUniformImage
from matplotlib import colors
from scipy.stats import norm
from matplotlib import cm as cm
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import logging
#from iopath.common.file_io import g_pathmgr, file_lock
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from matplotlib.gridspec import GridSpec
from sklearn.manifold import TSNE

In [5]:
import numpy as np
container_plot = 255 * np.ones((200, 300, 3), np.uint8)
np.shape(container_plot)

(200, 300, 3)

## discrete model

In [2]:
# info on specific run
year = '2015'
months = 'may-sep'
region = 'alps'
cropsize = 128
n_classes = 12

# read csv file with all information
run_info = f'{year}_{months}_{region}_{cropsize}_k{n_classes}'
csvfile = f'/p/scratch/deepacf/kiste/DC/dataset/paula/run_network/{run_info}/{run_info}_physical_properties.csv'
df = pd.read_csv(csvfile)
idx_12 = np.argwhere(df.label.values == 11)[:,0]

# define output path for plots
output_feature_space = '/p/scratch/deepacf/kiste/DC/dataset/paula/investigate_results/feature_space/'
output_example_imgs = '/p/scratch/deepacf/kiste/DC/dataset/paula/investigate_results/example_imgs/'

[4988.058  3235.6177 1779.6532 ... 2393.9983 3463.1587 2403.4485]


In [ ]:
# define colors for classes
Dict_classes = {'color_12':'#B8000F',
                'name_12':'ABL 1',
                'color_4':'#D53A47',
                'name_4':'ABL 2',
                'color_5':'#F37C86',
                'name_5':'ABL 3',
                'color_6':'#93C9EF',
                'name_6':'thick cirrus',
                'color_3':'#B0D9F6',
                'name_3':'thin cirrus',
                'color_2':'#000000',
                'name_2':'transition cloud',
                'color_10':'#D5DEE5',
                'name_10':'mixed phase 2',
                'color_8':'#A1A7AC',
                'name_8':'mixed phase 1',
                'color_7':'#707579',
                'name_7':'mixed phase 3',
                'color_9':'#5790B4',
                'name_9':'convective 1',
                'color_11':'#026BAC',
                'name_11':'convective 2',
                'color_1':'#059DFC',
                'name_1':'convective 3',
               }

# aspect ratio of all plots
aspect_ratio = 1.6

# sort the class labels according to order in Dict_classes
sort_classes = [k.split('_')[1] for k in Dict_classes if 'color' in k]


In [117]:
def _format_legend_feature_space(ax, dummy_data=False):
    if dummy_data:
        for str_label in sort_classes:
            ax.scatter(0,0, label=f"{Dict_classes[f'name_{str_label}']} ({str_label})", \
                       alpha=0.0, color=Dict_classes[f'color_{str_label}'])
    
    # setting up legend
    lgnd = plt.legend(bbox_to_anchor=(1.01, 0.5), loc="center left", borderaxespad=0, fontsize=20)
    for handle in lgnd.legendHandles:
        handle.set_sizes([60.0])
        handle.set_alpha(1)

def _compute_plot_coordinates(image, tsne_x, tsne_y, width_canvas, height_canvas, offset):
    image_size = image.shape[0]

    # compute the image center coordinates on the plot
    center_x = int(width_canvas * tsne_x) + offset

    # in matplotlib, the y axis is directed upward
    # to have the same here, we need to mirror the y coordinate
    center_y = int(height_canvas * (1-tsne_y)) + offset

    # knowing the image center, compute the coordinates of the top left corner
    tl_x = center_x - int(image_size / 2)
    tl_y = center_y - int(image_size / 2)
    # and bottom right corner
    br_x = tl_x + image_size
    br_y = tl_y + image_size
    
    return tl_x, tl_y, br_x, br_y

def _scale_image(image, max_image_size):
    image_size = image.shape[0]

    scale = max(1, image_size / max_image_size)
    image_scaled = int(image_size / scale)

    image = cv2.resize(image, (image_scaled, image_scaled))
    return image

def _draw_rectangle_by_class(image, str_label):
    image_size = image.shape[0]
    frame_size = int(image_size / 20)
    
    # get the color corresponding to image class
    color_rgb = PIL.ImageColor.getcolor(Dict_classes[f'color_{str_label}'], "RGB")
    
    # draw frame on image
    image = cv2.rectangle(image, (0, 0), (image_size - 1, image_size - 1), color=color_rgb, thickness=frame_size)
    return image

In [118]:
##### plot feature space with dots #####
########################################

# sample images without those that weren't assigned to a class (label=-100)
n_dots = 20000
df_subset = df.query("label != -100").sample(n=n_dots)
sort_classes = [k.split('_')[1] for k in Dict_classes if 'color' in k]

"""
# plot simple feature space
image_height = 10
fig = plt.figure(figsize=(aspect_ratio*image_height,image_height))
ax = fig.add_subplot(111)

plt.scatter(df_subset['tsne-x'],df_subset['tsne-y'],alpha=0.5,s=20)
ax.set_axis_off()
plt.savefig(output_path + f'{run_info}_feature_space_dots.png')
"""

# plot color coded feature space
fig_height = 20
fig = plt.figure(figsize=(aspect_ratio*fig_height,fig_height))
ax = fig.add_subplot(111)

for str_label in sort_classes:
    idx = np.argwhere(df_subset.label.values == int(str_label)-1)[:, 0]
    ax.scatter(df_subset['tsne-x'].values[idx], df_subset['tsne-y'].values[idx], 
               label=f"{Dict_classes[f'name_{str_label}']} ({str_label})", 
               alpha=0.5,s=int(fig_height/2)**2, color=Dict_classes[f'color_{str_label}'])

#_format_axes_feature_space(ax)
ax.set_axis_off()
_format_legend_feature_space(ax)
plt.savefig(output_feature_space + f'{run_info}_feature_space_dots_classes.png',transparent=True,bbox_inches='tight')

In [129]:
##### plot images in feature space #####
########################################

n_images = 400
images_subset = df.query("label != -100").sample(n=n_images)

figure_height = 15
figure_resolution = 100
fig = plt.figure(figsize=(aspect_ratio*figure_height,figure_height))
ax = fig.add_subplot(111)

# define plot and canvas size
plot_height = figure_height * figure_resolution
plot_width = int(aspect_ratio*plot_height)
max_image_size = int(figure_height * figure_resolution / 12)
offset = max_image_size // 2
width_canvas = plot_width - 2 * offset
height_canvas = plot_height - 2 * offset

# container to add the small images in
container_plot = 255 * np.ones((plot_height, plot_width, 3), np.uint8)

for index, row in images_subset.iterrows():
    img = row['file_location']
    str_label = f"{row['label']+1}"
    tsne_x = row['tsne-x']
    tsne_y = row['tsne-y']

    image = cv2.imread(img)[:, :, ::-1]
    image = _scale_image(image, max_image_size)  
    image = _draw_rectangle_by_class(image, str_label)
    tl_x, tl_y, br_x, br_y = _compute_plot_coordinates(image, tsne_x, tsne_y, width_canvas, height_canvas, offset)
    
    try:
        # tsne_plot[tl_y:br_y, tl_x:br_x, :] = image
        container_plot[tl_y:br_y, tl_x:br_x, :] = image
        
    except ValueError:
        print('could not plot image: ', img)
        continue

ax.imshow(container_plot)
_format_legend_feature_space(ax, dummy_data=True)
ax.set_axis_off()
plt.savefig(output_feature_space + f'{run_info}_feature_space_images_classes.png',transparent=True,bbox_inches='tight')
print('plot saved')

plot saved


In [58]:
##### plot example images per class #####
#########################################

n_per_class = 7

example_imgs = np.array([])
for s, str_label in enumerate(sort_classes):
    class_subset = np.array(df.query(f"label == {int(str_label)-1}").sample(n=n_per_class).file_location, dtype=str)
    example_imgs = np.append(example_imgs, class_subset)
example_imgs = np.reshape(example_imgs, (n_classes, n_per_class))

# set up figure
figure_scale = n_classes/n_per_class
fig = plt.figure(figsize=(n_classes*figure_scale, (n_per_class+0.1)*figure_scale),constrained_layout=True)
gs0 = fig.add_gridspec(1, 1)
gs00 = gs0[0].subgridspec(n_per_class, n_classes)

for k, str_class in enumerate(sort_classes):
    for i in range(n_per_class):
        ax = fig.add_subplot(gs00[i, k])
        file = example_imgs[k, i]
        image = plt.imread(file)
        image = _draw_rectangle_by_class(image, str_class)
        ax.imshow(image)
        ax.set_axis_off()
        
        if i == 0:
            ax.set_title(Dict_classes[f'name_{str_class}'], fontsize=figure_scale*8, color=Dict_classes[f'color_{str_class}'])

plt.savefig(output_example_imgs + f'{run_info}_{n_per_class}images_classes.png',transparent=True,bbox_inches='tight')